In [51]:
import numpy as np
import pandas as pd
import re as re
import lxml.html
import lxml.html.clean
from keras.preprocessing.text import Tokenizer
from string import digits
import nltk
# nltk.download('words')

# from nltk.tokenize import RegexpTokenizer
# from nltk.corpus import stopwords

token = RegexpTokenizer("[\w']+")
sw = stopwords.words('english')
words = set(nltk.corpus.words.words())

#reading training data from csv files
columnLabels = ['country', 'id', 'title', 'category-1', 'category-2', 'category-3', 'description', 'price', 'type']
train = pd.read_csv('data/training/data_train.csv', names=columnLabels, encoding = 'utf8')
clarity = pd.read_csv('data/training/clarity.csv', names=['clarity'], encoding = 'utf8')
concise = pd.read_csv('data/training/concise.csv', names=['concise'], encoding = 'utf8')
train['clarity'] = clarity
train['concise'] = concise


train.head()
cleandata(train)


[nltk_data] Downloading package words to C:\Users\OWAIS
[nltk_data]     SULTAN\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


,country,id,title,category-1,category-2,category-3,description,price,type,clarity,concise
1,my,AE068HBAA3RPRDANMY,heartbreaker de spray,Health & Beauty,Bath & Body,Hand & Foot Care,Formulated with oil-free hydrating botanicals/...,128.00,international,1,1
2,my,AN680ELAA9VN57ANMY,dual headed directional microphone collar clip...,"TV, Audio / Video, Gaming & Wearables",Audio,Live Sound & Stage,"150cm mini microphone compatible for iPhone, ...",25.07,international,1,0
3,my,AN957HBAAAHDF4ANMY,complaint silky set shampoo conditioner,Health & Beauty,Hair Care,Shampoos & Conditioners,ANMYNA Complaint Silky Set (Shampoo 520ml + C...,118.00,local,1,1
4,my,AR511HBAXNWAANMY,green clay face body,Health & Beauty,Men's Care,Body and Skin Care,100% Authentic Rrefresh and brighten skin Ant...,114.80,international,1,1
5,my,AS575ELCMZ4WANMY,transformer book flip core inch free care warr...,Computers & Laptops,Laptops,Traditional Laptops,Genuine Windows® 8.1 13.3in 16:9 HD TOUCH I...,2599.00,local,1,1
6,my,AS727ELAA9LLV1ANMY,ring shaped macro photography light circle rin...,Cameras,Camera Accessories,Lighting & Studio Equipment,1. Color Temperature: 5400K 2. Ra: >90 3. Vol...,388.99,international,1,1
7,my,BU512HBAA4WUVTANMY,peel foot mask,Health & Beauty,Bath & Body,Hand & Foot Care,Reviving like a new born baby. Age:Apply at a...,10.40,international,1,1
8,my,CL787ELAW29LANMY,slim flat extension cable,Computers & Laptops,Laptops,Traditional Laptops,Designed and constructed to super-speed USB ...,29.00,local,1,1
9,my,CO633HLAABREKOANMY,coke glass limited edition purple color,Home & Living,Kitchen & Dining,Tableware,Genuine issued McDonald's Coca Cola merchandi...,25.00,local,1,1
10,my,EL802HLAA51ZZVANMY,stainless steel tea ball strainer mesh infuser...,Home & Living,Kitchen & Dining,Cookware,Stainless Steel Filter Stainless Tea Filter T...,9.48,international,1,1


In [ ]:
#html tags removing from description
def remove_tags(string):
    doc = lxml.html.fromstring(string)
    cleaner = lxml.html.clean.Cleaner(style=True)
    doc = cleaner.clean_html(doc)
    text = doc.text_content()
    return text

def cleandata(data):
    
    #html tags removal
    data['description']=data['description'].apply(lambda cw : remove_tags(str(cw)))
    
    #tokenize and convert to lower case
    data['title'] = data['title'].apply(lambda x: token.tokenize(x.lower()))
    
    #stopword removal
    data['title'] = data['title'].apply(lambda x: ' '.join([w for w in x if w not in sw]))
    
    #remove punctuations and other symbols
    data['title'] = data['title'].apply(lambda x: [w for w in nltk.wordpunct_tokenize(str(x))\
                                                                 if w in words is not w.isalpha() and not w.isnumeric()])
    #remove digits
    data['title'] = data['title'].apply(lambda x: [w.translate({ord(k): None for k in digits}) for w in x])
    
    #remove single length words
    data['title'] = data['title'].apply(lambda x: ' '.join([w for w in x if len(w)>1]))

    
